# Demographics
This script is used for parsing demographics data.

Source: https://www.czso.cz/csu/czso/databaze-demografickych-udaju-za-obce-cr

In [4]:
import time
import pandas as pd
import numpy as np
import re
import json
import warnings
import json
import os

warnings.filterwarnings('ignore')


In [10]:
PATH_INTERMEDIATE = '../../data/intermediate/'
PATH_RAW = '../../data/raw/'
PATH_POPULATION = PATH_RAW+'population/'

In [77]:
demo = pd.DataFrame()
for name in os.listdir(PATH_POPULATION):
    demo = demo.append(pd.read_excel(PATH_POPULATION + name))

In [78]:
demo = demo[demo.Rok > 2009]
demo = demo[['Číslo\nobce', 'Název obce', 'Rok', 'Stav 31.12.', 'Narození', 'Zemřelí', 'Přírůstek přirozený', 'Přistě-\nhovalí', 'Vystě-\nhovalí', 'Přírůstek migrační', 'Přírůstek celkový']]
demo.columns = ['village_num', 'village_name', 'year', 'population', 'born', 'deceased', 'born_deceased', 'immigrants', 'emigrants', 'migration', 'change_total']

,village_num,village_name,year,population,born,deceased,born_deceased,immigrants,emigrants,migration,change_total
39,539104,Bojanovice,2010,425,5,8,-3,24,12,12,9
40,539104,Bojanovice,2011,420,6,12,-6,17,10,7,1
41,539104,Bojanovice,2012,417,1,4,-3,18,18,-,-3
42,539104,Bojanovice,2013,432,5,-,5,23,13,10,15
43,539104,Bojanovice,2014,447,2,6,-4,25,6,19,15


In [85]:
demo = demo[demo.population !='-']
demo = demo.applymap(lambda x: 0 if '-' == str(x) else x)

nums = demo.columns[2:]
demo[nums] = demo[nums].applymap(int)
demo

,village_num,village_name,year,population,born,deceased,born_deceased,immigrants,emigrants,migration,change_total
39,539104,Bojanovice,2010,425,5,8,-3,24,12,12,9
40,539104,Bojanovice,2011,420,6,12,-6,17,10,7,1
41,539104,Bojanovice,2012,417,1,4,-3,18,18,0,-3
42,539104,Bojanovice,2013,432,5,0,5,23,13,10,15
43,539104,Bojanovice,2014,447,2,6,-4,25,6,19,15
...,...,...,...,...,...,...,...,...,...,...,...
3144,585050,Zaječí,2017,1442,18,11,7,27,29,-2,5
3145,585050,Zaječí,2018,1432,19,20,-1,17,26,-9,-10
3146,585050,Zaječí,2019,1424,13,14,-1,34,41,-7,-8
3147,585050,Zaječí,2020,1454,15,13,2,61,33,28,30


In [94]:
# demo[demo.population.max() == demo.population]
demo21 = demo[demo.year == 2021]
demo21.groupby('population').max()
# demo[demo.village_name == 'Strančice']

,village_num,village_name,year,born,deceased,born_deceased,immigrants,emigrants,migration,change_total
population,,,,,,,,,,
16,561771,Vysoká Lhota,2021,0,0,0,2,0,2,2
20,540927,Čilá,2021,0,0,0,1,0,1,1
24,530964,Bludov,2021,1,2,-1,0,0,0,-1
28,529524,Čejkovice,2021,1,0,1,0,3,-3,-2
29,532932,Hradiště,2021,1,1,0,2,1,1,1
...,...,...,...,...,...,...,...,...,...,...
102951,563889,Liberec,2021,1070,1329,-259,3206,2434,772,513
168733,554791,Plzeň,2021,1751,2342,-591,5510,4451,1059,468
279791,554821,Ostrava,2021,2885,4256,-1371,4594,5697,-1103,-2474


In [98]:
demo.to_csv(PATH_INTERMEDIATE+'demographics.csv', index=False)